# Machine Learning Homework 5
\> БИБ201 Рудзянский Артемий 

## Кластеризация: О вкусах не спорят
**Задача**
- Создание своего датасета  
        features: text  
        label: artist
    - Очистка текста от знаков препинания
    - Использование Encoder
- Кластеризация полученного датасета
- Анализ результатов

## Создание своего датасета
Выбранная структура датасета была представлена выше.

Я решил использовать публичный API для получения текстов песен.

Процесс:
1) Я выбрал несколько артистов разных жанров (с целью, чтобы их тексты больше различались)
2) Получил список песен выбранных исполнителей
2) Через [ChartLyrics Lyric API](http://www.chartlyrics.com/api.aspx) получил тексты песен 

0) Eminem  
1) Philip Wesley  
2) NEFFEX  
3) Ed Sheeran  
4) The Beatles  
5) System of A Down  
6) Gorillaz  
7) Toto  
8) TOOL  
9) Nightwish 

In [11]:
import numpy as np
import pandas as pd
import requests
try:
    import xmltodict
except ImportError as e:
    !pip install xmltodict
    import xmltodict
from sklearn.feature_extraction.text import TfidfVectorizer
np.random.seed(42)
from sklearn.cluster import KMeans

# to plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

### Демонстрация обработки изначального csv файла на примере *Eminem.csv
Последующие csv файлы будут обработаны аналогично

Для начала напишем функцию получения текста песни при помощи публичного API
Также напишем функцию, чистящую полученный текст, оставляя только одинарные пробелы и сам текст заглавными буквами

In [2]:
# Функция получения текста песни
def getLyrics(artist, song):
    # Параметры запроса
    url = "http://api.chartlyrics.com/apiv1.asmx/SearchLyricDirect"

    querystring = {"artist": artist, "song": song}

    headers = {
    	"X-RapidAPI-Key": "SIGN-UP-FOR-KEY",
    	"X-RapidAPI-Host": "sridurgayadav-chart-lyrics-v1.p.rapidapi.com"
    }
    # Выполняем запрос
    try:
        response = requests.request("GET", url, headers=headers, params=querystring)
        response.raise_for_status()
    except requests.exceptions.HTTPError as err:
        raise ValueError("Bad Response")

    # Обрабатываем ответ
    dict_data = xmltodict.parse(response.content)['GetLyricResult']
    if "LyricSong" in dict_data:
        if dict_data["LyricArtist"].upper() == artist.upper() and dict_data["LyricSong"].upper() == song.upper() and dict_data["Lyric"] is not None:
            lyrics = dict_data["Lyric"]
            return clean_lyrics(lyrics) 
        else:
            raise ValueError(f"Not found exact song {song}, {artist}")
    else:
        raise ValueError("Not found anything about the song")


# Функция очищения текста от лишних символов
check_letter = lambda c: (64 < ord(c) and ord(c) < 91) or ord(c) == 32
def clean_lyrics(lyrics):
    lyrics = lyrics.upper()
    #lyrics = ''.join(list(filter(check_letter, list(lyrics))))
    lyrics = ''.join(list(filter(check_letter, list(lyrics))))

    return ' '.join(lyrics.split())

In [3]:
# df = pd.read_csv("Eminem.csv", names=['Track', 'Artist'], header=0, usecols=['Track', 'Artist'])
# df.head()
# idx = df.index[df["Track"] == "Bad Guy"].tolist()
# print(idx)
# df.at[idx[0], 'Track'] = ['q', 'z']
# display(df)
#df[df["Track"] == 'Bad Guy']["Artist"] = ["q", "z"]

In [4]:
eminem_df = pd.read_csv("Eminem.csv", names=['Track', 'Artist'], header=0, usecols=['Track', 'Artist'])
eminem_df = eminem_df[eminem_df["Artist"] == "Eminem"] # remain only given artist
display(eminem_df.head(), eminem_df.shape)
#df_text = pd.DataFrame(np.nan, index=range(eminem_df.shape[0]), columns=['Text'])
#display(type(df_text))

#eminem_df = pd.concat([eminem_df, df_text], axis=1)
for track in eminem_df["Track"].values:
    try:
        lyrics = getLyrics("Eminem", track)
        # put lyrics instead a name of a track
        idx = eminem_df.index[eminem_df["Track"] == track].tolist()[0]
        eminem_df.at[idx, 'Track'] = lyrics
    except ValueError: # delete track if we didn't find lyric
        eminem_df = eminem_df[eminem_df['Track'] != track]
display(eminem_df.head(), eminem_df.shape)

,Track,Artist
0,"Lose Yourself - From ""8 Mile"" Soundtrack",Eminem
1,The Real Slim Shady,Eminem
2,Stan,Eminem
3,Till I Collapse,Eminem
4,My Name Is,Eminem


(52, 2)

,Track,Artist
1,MAY I HAVE YOUR ATTENTION PLEASEMAY I HAVE YOU...,Eminem
4,HI MY NAME IS WHATMY NAME IS WHOMY NAME IS CHI...,Eminem
7,INTROWHERES MY SNAREI HAVE NO SNARE ON MY HEAD...,Eminem
8,NOT AFRAIDCHORUSIM NOT AFRAID IM NOT AFRAIDTO ...,Eminem
9,MAN WHATEVERDRE JUST LET IT RUNAYO TURN THE BE...,Eminem


(16, 2)

Выше был представлен наглядный пример превращения начального csv файла в готовый кусок датасета  
Теперь же мы напишем функцию, которая позволит перебрать всех артистов через цикл

In [5]:
def from_csv_to_df(artist):
    df = pd.read_csv(f"{artist}.csv", names=['Track', 'Artist'], header=0, usecols=['Track', 'Artist'])
    df = df[df['Artist'] == artist]
    for track in df['Track'].values:
        try:
            lyrics = getLyrics(artist, track)
            # put lyrics instead a name of a track
            idx = df.index[df["Track"] == track].tolist()[0]
            df.at[idx, 'Track'] = lyrics
           #df = df.replace(to_replace=track, value=lyrics, inplace=True)
        except ValueError as err:
            # print(err)
            df = df[df['Track'] != track]
    return df

Выпишем интересующих нас артистов в виде листа

In [6]:
arr_artists = [
    # 'Eminem',  
    'Elton John', 
    'Michael Jackson',
    'The Beatles',
    'System of A Down',
    'Gorillaz', 
    'TOTO', 
    'TOOL', 
    'Nightwish'
]

df = eminem_df
for artist in arr_artists:
    df_new = from_csv_to_df(artist)
    df = pd.concat([df, df_new])


In [7]:
df = df.loc[:, ["Artist", "Track"]]
display(df.shape, df.head())
df.to_csv("out.csv", index=None)

(186, 2)

,Artist,Track
1,Eminem,MAY I HAVE YOUR ATTENTION PLEASEMAY I HAVE YOU...
4,Eminem,HI MY NAME IS WHATMY NAME IS WHOMY NAME IS CHI...
7,Eminem,INTROWHERES MY SNAREI HAVE NO SNARE ON MY HEAD...
8,Eminem,NOT AFRAIDCHORUSIM NOT AFRAID IM NOT AFRAIDTO ...
9,Eminem,MAN WHATEVERDRE JUST LET IT RUNAYO TURN THE BE...


Датасет готов

In [38]:
vectorizer = TfidfVectorizer(max_df=0.2)
X = vectorizer.fit_transform(df["Track"]).toarray()

In [47]:
display(len(X), df.shape, len(vectorizer.get_feature_names_out()))
display(X.shape, X[20].shape)

186

(186, 2)

8726

(186, 8726)

(8726,)

In [12]:
kmeans = KMeans(n_clusters=8, random_state=42).fit(X)

In [13]:
clusters = kmeans.labels_

In [14]:
display(clusters)

array([3, 3, 3, 7, 3, 3, 7, 7, 1, 7, 1, 3, 7, 3, 7, 3, 1, 3, 3, 7, 7, 3,
       7, 3, 1, 3, 3, 3, 3, 3, 7, 1, 3, 6, 3, 5, 6, 1, 3, 3, 5, 3, 7, 7,
       5, 3, 1, 7, 4, 1, 6, 0, 6, 7, 4, 0, 6, 0, 1, 1, 3, 3, 4, 1, 4, 5,
       7, 1, 3, 7, 4, 3, 7, 3, 1, 3, 3, 6, 3, 4, 7, 7, 7, 6, 1, 3, 1, 3,
       7, 6, 7, 7, 4, 3, 0, 1, 7, 3, 7, 7, 4, 3, 5, 3, 1, 1, 3, 3, 3, 1,
       7, 7, 3, 7, 3, 1, 2, 1, 7, 4, 7, 3, 7, 5, 7, 5, 2, 4, 7, 3, 7, 7,
       3, 1, 1, 7, 6, 7, 7, 7, 7, 3, 3, 7, 3, 7, 7, 7, 7, 3, 1, 7, 7, 3,
       3, 2, 5, 6, 4, 7, 7, 7, 7, 3, 7, 7, 3, 1, 7, 7, 3, 3, 3, 1, 7, 7,
       3, 7, 3, 1, 7, 3, 7, 3, 3, 1], dtype=int32)

Получили:
3 - Eminem

In [16]:
print(df["Artist"].values)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df["Artist"].values)

['Eminem' 'Eminem' 'Eminem' 'Eminem' 'Eminem' 'Eminem' 'Eminem' 'Eminem'
 'Eminem' 'Eminem' 'Eminem' 'Eminem' 'Eminem' 'Eminem' 'Eminem' 'Eminem'
 'Elton John' 'Elton John' 'Elton John' 'Elton John' 'Elton John'
 'Elton John' 'Elton John' 'Elton John' 'Elton John' 'Elton John'
 'Elton John' 'Elton John' 'Elton John' 'Elton John' 'Elton John'
 'Elton John' 'Michael Jackson' 'Michael Jackson' 'Michael Jackson'
 'Michael Jackson' 'Michael Jackson' 'Michael Jackson' 'Michael Jackson'
 'Michael Jackson' 'Michael Jackson' 'Michael Jackson' 'Michael Jackson'
 'Michael Jackson' 'Michael Jackson' 'Michael Jackson' 'Michael Jackson'
 'Michael Jackson' 'Michael Jackson' 'Michael Jackson' 'Michael Jackson'
 'The Beatles' 'The Beatles' 'The Beatles' 'The Beatles' 'The Beatles'
 'The Beatles' 'The Beatles' 'The Beatles' 'The Beatles' 'The Beatles'
 'The Beatles' 'The Beatles' 'The Beatles' 'The Beatles' 'The Beatles'
 'The Beatles' 'The Beatles' 'The Beatles' 'The Beatles' 'The Beatles'
 'The Beatle

In [59]:
from itertools import permutations
import itertools
print(arr_artists)
perms = list(permutations(range(8)))
for perm in perms:
    for i in perm:
        idx = df.index[df["Artist"] == arr_artists[i]].tolist()[0]
        print(idx)
        # df.at[idx, 'Track'] = lyrics
        

import seaborn as sns
sns.acatter 

['Elton John', 'Michael Jackson', 'The Beatles', 'System of A Down', 'Gorillaz', 'TOTO', 'TOOL', 'Nightwish']
3
0
3


IndexError: list index out of range

Я считаю, что разброс сета не имеет кластерную структуру. Так как большинство ходовых слов встречается во всех текстах вне зависимости от жанра и артиста. И даже всё равно после того как мы нормализовали частоту попадания популярных слов при помощи Vectorizer, проблема простоты модели никуда не подевалась. Также слишком малые значения в матрице при tf-idf
Здесь стоило бы использовать анализ словосочетаний. Это бы намного сильнее увеличило эффективность.
Если работать только с ключевыми словами, то будет работать лучше.